In [1]:
import matplotlib.pyplot as plt

In [2]:
import sys
from os.path import dirname
sys.path.append(dirname('./synthcity/src/'))

In [3]:
import torch
import torch.nn as nn
import numpy as np

C:\Users\ma7mo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import torch.optim as optim
from torch.utils.data import DataLoader

In [6]:
import seaborn as sns

In [7]:
from synthcity.plugins import Plugins
from synthcity.benchmark import Benchmarks

[KeOps] Warning : 
    The default C++ compiler could not be found on your system.
    You need to either define the CXX environment variable or a symlink to the g++ command.
    For example if g++-8 is the command you can do
      import os
      os.environ['CXX'] = 'g++-8'
    
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [8]:
Plugins(categories=["time_series"]).list()

['timegan', 'fflows', 'timevae']

In [9]:
from synthcity.utils.datasets.time_series.pbc import PBCDataloader
from synthcity.utils.datasets.time_series.google_stocks import GoogleStocksDataloader
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader

In [10]:
static_data, temporal_data, horizons, outcome = GoogleStocksDataloader().load()

In [11]:
data = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=horizons,
    static_data=static_data,
    outcome=outcome,
)

In [12]:
syn_model_without_dp = Plugins().get("timegan", dp_enabled=False)

[2023-08-29T12:31:13.236979-0400][7524][CRITICAL] module disabled: C:\Users\ma7mo\AppData\Local\Programs\Python\Python310\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py


In [13]:
new_model = Plugins().get("timegan", mode="LSTM")

[2023-08-29T12:31:13.757252-0400][7524][CRITICAL] module disabled: C:\Users\ma7mo\AppData\Local\Programs\Python\Python310\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py
[2023-08-29T12:31:13.757252-0400][7524][CRITICAL] module disabled: C:\Users\ma7mo\AppData\Local\Programs\Python\Python310\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py


In [14]:
new_model.fit(data)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [15:01<00:00,  1.11it/s]


In [15]:
syn_model_without_dp.fit(data)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:56<00:00,  4.22it/s]


In [16]:
synth_data = syn_model_without_dp.generate(count=len(data)/10)

In [17]:
synth_data

,seq_id,seq_time_id,seq_temporal_Close,seq_temporal_High,seq_temporal_Low,seq_temporal_Open,seq_temporal_Volume,seq_out_Open_next
0,0,0.997485,0.402246,0.347664,0.401535,0.373814,0.213604,0.314527
1,0,0.975117,0.417518,0.347480,0.412731,0.373461,0.215196,0.314527
2,0,0.988041,0.421510,0.347617,0.405933,0.373382,0.205918,0.314527
3,0,0.942304,0.422826,0.372049,0.399245,0.364699,0.203831,0.314527
4,0,0.926014,0.414588,0.375125,0.402810,0.373350,0.195506,0.314527
...,...,...,...,...,...,...,...,...
495,49,0.745865,0.422670,0.356128,0.410471,0.387060,0.192658,0.314527
496,49,0.765601,0.421731,0.362003,0.421643,0.387181,0.211674,0.314527
497,49,0.833377,0.425472,0.366295,0.428284,0.190931,0.243209,0.314527
498,49,0.874923,0.427297,0.356959,0.277438,0.196516,0.220372,0.314527


In [18]:
def create_sequence(dataset, seq_len):
    sequences = []
    labels = []


    for index in range(0, len(dataset)//seq_len): # Selecting 50 rows at a time
        sequences.append(dataset.iloc[index*seq_len:(index+1)*seq_len-1])
        labels.append(dataset.iloc[(index+1)*seq_len-1][2])
    return (np.array(sequences),np.array(labels))

In [31]:
class Network(nn.Module):
    def __init__(self, dropout,
                 input_dim=8, num_classes = 1, hidden_size = 50, **kwargs):

        super(Network, self).__init__()

        print("==> not used params in network class:", kwargs.keys())

        self.input_dim = input_dim
        self.dropout = dropout
        self.hidden_size = hidden_size


        # Main part of the network
        self.lstm_layers = nn.ModuleList()
        self.lstm_layers.append(nn.LSTM(input_size=input_dim,hidden_size=self.hidden_size))
        self.lstm_layers.append(nn.Dropout(p=dropout))
        self.lstm_layers.append(nn.LSTM(input_size=self.hidden_size,
                           hidden_size=self.hidden_size))
        self.lstm_layers.append(nn.Dropout(p=dropout))
        self.lstm_layers.append(nn.LSTM(input_size=self.hidden_size,
                           hidden_size=self.hidden_size))
        self.lstm_layers.append(nn.Dropout(p=dropout))
        # Output layer
        self.output_layer = nn.Linear(self.hidden_size, num_classes)
        # self.output_activation = nn.Sigmoid()

    def forward(self, X, lengths=None):
        # Input layer
        X = X.to(torch.float32)
        mX = X
        # Main part of the network
        i = 0
        for lstm in self.lstm_layers:
            if i % 2 == 0:
                output, _ = lstm(mX)
            else:
                output = lstm(mX)
            mX = output
            i += 1
        L = output[:, -1, :]
        # Output layer
        y = self.output_layer(L)
        return y


In [40]:
model = Network(0.2)

==> not used params in network class: dict_keys([])


In [41]:
optimizer_config = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
loss = nn.MSELoss()

In [42]:
X, y = create_sequence(data.dataframe(), 10)

In [43]:
synth_X, synth_y = create_sequence(synth_data.dataframe(), 10)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [45]:
train_loader = DataLoader(list(zip(X_train, y_train)), batch_size=1)
val_loader = DataLoader(list(zip(X_test, y_test)), batch_size=1, shuffle=True)
optimizer = optimizer_config

In [46]:
for epoch in range(0, 100):
        model.train()
        running_loss = 0
        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            outputs = model(inputs)
            targets = targets.reshape(tuple(outputs.shape))
            targets = targets.to(torch.float32)
            batch_loss = loss(outputs, targets)
            batch_loss.backward()
            optimizer.step()
        model.eval()
        val_loss = 0
        with torch.no_grad():
            num_batches = 0
            res = {}
            y_true = []
            predictions = []
            for batch in val_loader:
                inputs, targets = batch
                outputs = model(inputs)
                targets = targets.reshape(tuple(outputs.shape))
                targets = targets.to(torch.float32)
                batch_loss = loss(outputs, targets)
                val_loss += batch_loss.item()
                num_batches += 1
            val_loss /= num_batches
            print('Epoch [{}/{}], Step [{}/{}], Validation Batch Loss: {:.4f}'
                    .format(epoch + 1, 100, i + 1, len(train_loader), val_loss))

Epoch [1/100], Step [33/33], Validation Batch Loss: 0.1216
Epoch [2/100], Step [33/33], Validation Batch Loss: 0.0510
Epoch [3/100], Step [33/33], Validation Batch Loss: 0.0497
Epoch [4/100], Step [33/33], Validation Batch Loss: 0.0497
Epoch [5/100], Step [33/33], Validation Batch Loss: 0.0501
Epoch [6/100], Step [33/33], Validation Batch Loss: 0.0505
Epoch [7/100], Step [33/33], Validation Batch Loss: 0.0507
Epoch [8/100], Step [33/33], Validation Batch Loss: 0.0507
Epoch [9/100], Step [33/33], Validation Batch Loss: 0.0498
Epoch [10/100], Step [33/33], Validation Batch Loss: 0.0493
Epoch [11/100], Step [33/33], Validation Batch Loss: 0.0469
Epoch [12/100], Step [33/33], Validation Batch Loss: 0.0436
Epoch [13/100], Step [33/33], Validation Batch Loss: 0.0427
Epoch [14/100], Step [33/33], Validation Batch Loss: 0.0376
Epoch [15/100], Step [33/33], Validation Batch Loss: 0.0337
Epoch [16/100], Step [33/33], Validation Batch Loss: 0.0310
Epoch [17/100], Step [33/33], Validation Batch Lo

In [47]:
synth_model = Network(0.2)

==> not used params in network class: dict_keys([])


In [48]:
optimizer_config = optim.Adam(synth_model.parameters(), lr=0.001, betas=(0.9, 0.999))
loss = nn.MSELoss()

In [49]:
train_loader = DataLoader(list(zip(synth_X, synth_y)), batch_size=1, shuffle=True)
val_loader = DataLoader(list(zip(X_test, y_test)), batch_size=1)
optimizer = optimizer_config

In [ ]:
for epoch in range(0, 100):
        synth_model.train()
        running_loss = 0
        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            outputs = synth_model(inputs)
            targets = targets.reshape(tuple(outputs.shape))
            targets = targets.to(torch.float32)
            batch_loss = loss(outputs, targets)
            batch_loss.backward()
            optimizer.step()
        synth_model.eval()
        val_loss = 0
        with torch.no_grad():
            num_batches = 0
            res = {}
            y_true = []
            predictions = []
            for batch in val_loader:
                inputs, targets = batch
                outputs = synth_model(inputs)
                targets = targets.reshape(tuple(outputs.shape))
                targets = targets.to(torch.float32)
                batch_loss = loss(outputs, targets)
                val_loss += batch_loss.item()
                num_batches += 1
            val_loss /= num_batches
            print('Epoch [{}/{}], Step [{}/{}], Validation Batch Loss: {:.4f}'
                    .format(epoch + 1, 100, i + 1, len(train_loader), val_loss))

Epoch [1/100], Step [50/50], Validation Batch Loss: 0.0809
Epoch [2/100], Step [50/50], Validation Batch Loss: 0.1036
Epoch [3/100], Step [50/50], Validation Batch Loss: 0.1216
Epoch [4/100], Step [50/50], Validation Batch Loss: 0.1078
Epoch [5/100], Step [50/50], Validation Batch Loss: 0.0825
Epoch [6/100], Step [50/50], Validation Batch Loss: 0.1011
Epoch [7/100], Step [50/50], Validation Batch Loss: 0.0882
Epoch [8/100], Step [50/50], Validation Batch Loss: 0.0731
Epoch [9/100], Step [50/50], Validation Batch Loss: 0.1061
Epoch [10/100], Step [50/50], Validation Batch Loss: 0.0838
Epoch [11/100], Step [50/50], Validation Batch Loss: 0.0630
Epoch [12/100], Step [50/50], Validation Batch Loss: 0.0566
Epoch [13/100], Step [50/50], Validation Batch Loss: 0.0379
Epoch [14/100], Step [50/50], Validation Batch Loss: 0.0152
Epoch [15/100], Step [50/50], Validation Batch Loss: 0.0129
Epoch [16/100], Step [50/50], Validation Batch Loss: 0.0147
Epoch [17/100], Step [50/50], Validation Batch Lo